In [42]:
#Test use of MS azure text analytics
import requests
#pprint for formatting JSON
from pprint import pprint as pp #hehehehehe
import os
import pandas as pd
from pandas.io.json import json_normalize
key = 'TEXT_ANALYTICS_SUBSCRIPTION_KEY'
if not key in os.environ:
    raise Exception("Please set/export the environment variable: {}".format(key))
sub_key = os.environ[key]

endpoint_name = 'TEXT_ANALYTICS_ENDPOINT'
if not endpoint_name in os.environ:
    raise Exception('Please set/export the environment variable: {}'.format(endpoint_name))
endpoint = os.environ[endpoint_name]

In [43]:
data = pd.read_csv('IRAhandle_tweets_subset.csv', encoding = 'latin')
data.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,9.058750e+17,9.145800e+17,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,9.058750e+17,9.146220e+17,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,9.058750e+17,9.146230e+17,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,9.058750e+17,9.146390e+17,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,9.058750e+17,9.143120e+17,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


'Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt should say Lynch vs. belt https://t.co/mLH1i30LZZ'

In [44]:
#keywords analysis
keyphrase_url = endpoint + "/text/analytics/v2.1/keyphrases"
def keyword(document):
    headers = {"Ocp-Apim-Subscription-Key": sub_key}
    response = requests.post(keyphrase_url, headers=headers, json=documents)
    key_phrases = response.json()
    words = key_phrases['documents']
    return words

In [45]:
#sentiment analysis
sentiment_url = endpoint + "/text/analytics/v2.1/sentiment"
def get_sentiment(document):
    headers = {"Ocp-Apim-Subscription-Key": sub_key}
    response = requests.post(sentiment_url, headers=headers, json=documents)
    sentiments = response.json()
    sent_val = sentiments['documents']
    return sent_val

In [75]:
#entity extraction
entities_url = endpoint + "/text/analytics/v2.1/entities"
def extract_entities(document):
    headers = {"Ocp-Apim-Subscription-Key": sub_key}
    response = requests.post(entities_url, headers=headers, json=documents)
    entities = response.json()
    entity_doc = entities['documents']
    return entity_doc

In [76]:
text = 'Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt should say Lynch vs. belt'
documents = {"documents": [{"id": "1", "language": "en","text": text}
                          ]}
sent = extract_entities(documents)
sent

[{'id': '1',
  'entities': [{'name': 'Marshawn Lynch',
    'matches': [{'entityTypeScore': 0.9930055141448975,
      'text': 'Marshawn Lynch',
      'offset': 0,
      'length': 14}],
    'type': 'Person'},
   {'name': 'Lynch',
    'matches': [{'entityTypeScore': 0.5752251148223877,
      'text': 'Lynch',
      'offset': 102,
      'length': 5}],
    'type': 'Person'}]}]

In [74]:
sent


In [64]:
authors = []
dates = []
sentiment = []
category = []
keywords = []
for idx,x in data.iterrows():
    text = x['content']
    documents = {"documents": [
    {"id": "1", "language": "en",
        "text": text}
    ]}
    words = keyword(documents)
    keyw = json_normalize(words)#df with flattened json
    sent = get_sentiment(documents)
    sents = json_normalize(sent)#df with flattened json
    authors.append(x['author'])
    dates.append(x['publish_date'])
    category.append(x['account_category'])
    sentiment.append(sents.score[0])
    keywords.append(keyw.keyPhrases[0])

In [65]:
output_df = pd.DataFrame()
output_df['author'] = authors
output_df['dates'] = dates
output_df['category'] = category
output_df['sentiment_value'] = sentiment
output_df['keywords'] = keywords

output_df

,author,dates,category,sentiment_value,keywords
0,10_GOP,10/1/2017 19:58,RightTroll,0.219764,"[corruption, trial, sitting Democrat US Senato..."
1,10_GOP,10/1/2017 22:43,RightTroll,0.718422,"[Marshawn Lynch, anti-Trump shirt, game, saggi..."
2,10_GOP,10/1/2017 22:50,RightTroll,0.796842,"[anthem protests, powerful monologue, Daughter..."
3,10_GOP,10/1/2017 23:52,RightTroll,0.500000,"[Presidents Cup golf tournament trophy, people..."
4,10_GOP,10/1/2017 2:13,RightTroll,0.500000,"[National Anthem, StandForOurAnthemðºð]"
5,10_GOP,10/1/2017 2:47,RightTroll,0.926685,"[liberals, Dan Bongino, Donald Trump]"
6,10_GOP,10/1/2017 2:48,RightTroll,0.752674,[ððð]
7,10_GOP,10/1/2017 2:52,RightTroll,0.085778,"[CNN, SenatorMenendez, CarmenYulinCruz, crimes..."
8,10_GOP,10/1/2017 3:47,RightTroll,0.221319,"[Trump, PR relief, days, CNN article]"
9,10_GOP,10/1/2017 3:51,RightTroll,0.268475,"[remark, San Juan Mayor, genocide, narrative, ..."


In [68]:
output_df.sentiment_value.describe()

count    50.000000
mean      0.539698
std       0.323538
min       0.028125
25%       0.220153
50%       0.500000
75%       0.797785
max       0.994611
Name: sentiment_value, dtype: float64

{'documents': [{'entities': [{'matches': [{'entityTypeScore': 0.8,
                                           'length': 9,
                                           'offset': 32,
                                           'text': '11-4-2019'}],
                              'name': '11-4-2019',
                              'subType': 'Date',
                              'type': 'DateTime'},
                             {'matches': [{'entityTypeScore': 0.9984753131866455,
                                           'length': 12,
                                           'offset': 45,
                                           'text': 'Ryan Huffman'}],
                              'name': 'Ryan Huffman',
                              'type': 'Person'},
                             {'bingId': 'cf3abf7d-e379-2693-f765-6da6b9fa9149',
                              'matches': [{'entityTypeScore': 0.8,
                                           'length': 15,
                               